In [442]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.5
Sklearn  0.24.2


In [443]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "umutakta"
    os.environ['KAGGLE_KEY']      = "a471e5474add8d57cadac3cc5ff2b1d6"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "API_(Day_6)/data/"

else:
    DATA_PATH = "/home/umut/Documents/GitHub/Strive_School/Full_Stack_Deep_Learning_(Module_8)/API_(Day_6)/data/"

DATA_PATH

'/home/umut/Documents/GitHub/Strive_School/Full_Stack_Deep_Learning_(Module_8)/API_(Day_6)/data/'

In [444]:
df      = pd.read_csv(DATA_PATH + "train.csv", )
df_test = pd.read_csv(DATA_PATH + "test.csv", )

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 12)
Test DataFrame:  (418, 11)


In [445]:
df = df.drop(['Ticket', "Cabin","PassengerId"], axis=1)
df_test = df_test.drop(['Ticket',"Cabin","PassengerId"], axis=1)

In [446]:

get_Title_from_Name = lambda x: x.split(",")[1].split(".")[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name )
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [447]:
combine = [df, df_test]


for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')





In [448]:
df["Title"] = df['Title'].map(title_dictionary)
df_test["Title"] = df_test['Title'].map(title_dictionary)

In [449]:
df = df.drop(['Name'], axis=1)
df_test = df_test.drop(['Name'], axis=1)


In [450]:
combine = [df, df_test]
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,S,Mr
1,1,1,1,38.0,1,0,71.2833,C,Mrs
2,1,3,1,26.0,0,0,7.9250,S,Miss
3,1,1,1,35.0,1,0,53.1000,S,Mrs
4,0,3,0,35.0,0,0,8.0500,S,Mr


In [451]:
guess_ages = np.zeros((2,3))
combine = [df, df_test]
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22,1,0,7.2500,S,Mr
1,1,1,1,38,1,0,71.2833,C,Mrs
2,1,3,1,26,0,0,7.9250,S,Miss
3,1,1,1,35,1,0,53.1000,S,Mrs
4,0,3,0,35,0,0,8.0500,S,Mr


In [452]:
df['AgeBand'] = pd.cut(df['Age'], 5)
df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(-0.08, 16.0]",0.550000
1,"(16.0, 32.0]",0.337374
2,"(32.0, 48.0]",0.412037
3,"(48.0, 64.0]",0.434783
4,"(64.0, 80.0]",0.090909


In [453]:
combine = [df, df_test]
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,0,3,0,1,1,0,7.2500,S,Mr,"(16.0, 32.0]"
1,1,1,1,2,1,0,71.2833,C,Mrs,"(32.0, 48.0]"
2,1,3,1,1,0,0,7.9250,S,Miss,"(16.0, 32.0]"
3,1,1,1,2,1,0,53.1000,S,Mrs,"(32.0, 48.0]"
4,0,3,0,2,0,0,8.0500,S,Mr,"(32.0, 48.0]"


In [454]:
df = df.drop(['AgeBand'], axis=1)
combine = [df, df_test]
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,1,1,0,7.2500,S,Mr
1,1,1,1,2,1,0,71.2833,C,Mrs
2,1,3,1,1,0,0,7.9250,S,Miss
3,1,1,1,2,1,0,53.1000,S,Mrs
4,0,3,0,2,0,0,8.0500,S,Mr


In [455]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1


In [456]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1



In [457]:

df = df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
df_test = df_test.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [df, df_test]

df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,1,7.2500,S,Mr,0
1,1,1,1,2,71.2833,C,Mrs,0
2,1,3,1,1,7.9250,S,Miss,1
3,1,1,1,2,53.1000,S,Mrs,0
4,0,3,0,2,8.0500,S,Mr,1


In [458]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass



In [459]:
freq_port = df.Embarked.dropna().mode()[0]
freq_port

'S'

In [460]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    


In [461]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [462]:
df_test['Fare'].fillna(df_test['Fare'].dropna().median(), inplace=True)
df_test.head()

,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,3,0,2,7.8292,2,Mr,1,6
1,3,1,2,7.0000,0,Mrs,0,6
2,2,0,3,9.6875,2,Mr,1,6
3,3,0,1,8.6625,0,Mr,1,3
4,3,1,1,12.2875,0,Mrs,0,3


In [463]:
df['FareBand'] = pd.qcut(df['Fare'], 4)

In [464]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

df = df.drop(['FareBand'], axis=1)
combine = [df, df_test]

In [468]:
x = df.drop(columns=["Survived"]) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test # # X_TEST DATA (NEW DATA)
x

,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,3,0,1,0,0,Mr,0,3
1,1,1,2,3,1,Mrs,0,2
2,3,1,1,1,0,Miss,1,3
3,1,1,2,3,0,Mrs,0,2
4,3,0,2,1,0,Mr,1,6
...,...,...,...,...,...,...,...,...
886,2,0,1,1,0,NaN,1,2
887,1,1,1,2,0,Miss,1,1
888,3,1,1,2,0,Miss,0,3
889,1,0,1,2,1,Mr,1,1


In [469]:
cat_vars=list(x.columns)
#print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)



Categorical features:
 ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Title', 'IsAlone', 'Age*Class']


In [488]:

cat_4_multModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('one hot', preprocessing.OneHotEncoder(handle_unknown="ignore"
))
])

mult_prepro = compose.ColumnTransformer(transformers=[

    ("cat mult",cat_4_multModels,cat_vars),

], remainder='drop')

mult_prepro



ColumnTransformer(transformers=[('cat mult',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('one hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked',
                                  'Title', 'IsAlone', 'Age*Class'])])

In [489]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [490]:
mult_classifiers={
    "SVM": svm.SVC(),
    "Logistic": LogisticRegression(),
    "KNN":KNeighborsClassifier(n_neighbors=3),
    "MLP":MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)}

mult_classifiers = {name: pipeline.make_pipeline(mult_prepro, model) for name, model in mult_classifiers.items()}

mult_classifiers["SVM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Pclass', 'Sex', 'Age',
                                                   'Fare', 'Embarked', 'Title',
                                                   'IsAlone', 'Age*Class'])])),
                ('svc', SVC())])

In [491]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in mult_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

,Model,Accuracy,Bal Acc.,Time
1,Logistic,84.357542,82.681159,0.038644
2,SVM,83.798883,80.876153,0.055426
3,MLP,81.005587,82.114625,0.229092
4,KNN,78.212291,78.221344,0.030334


In [511]:

cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('one hot', preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value=np.nan
))
])

tree_prepro = compose.ColumnTransformer(transformers=[

    ("cat mult",cat_4_treeModels,cat_vars),

], remainder='drop')

tree_prepro


ColumnTransformer(transformers=[('cat mult',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('one hot',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=nan))]),
                                 ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked',
                                  'Title', 'IsAlone', 'Age*Class'])])

In [512]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [513]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": GradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier() }


# YOUR CODE HERE


tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat mult',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one hot',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Pclass', 'Sex', 'Age',
                                                   'Fare', 'Embarked', 'Title',
                                                   'IsAlone', 'Age*Class'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [516]:

x_train, x_val, y_train, y_val = model_selection.train_test_split(x,y, test_size=0.2,stratify = y, random_state=4)




results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train,y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              





results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')


s	remaining: 272ms
618:	learn: 0.3428915	total: 441ms	remaining: 271ms
619:	learn: 0.3428816	total: 441ms	remaining: 270ms
620:	learn: 0.3428272	total: 442ms	remaining: 270ms
621:	learn: 0.3427919	total: 443ms	remaining: 269ms
622:	learn: 0.3427446	total: 443ms	remaining: 268ms
623:	learn: 0.3426695	total: 444ms	remaining: 268ms
624:	learn: 0.3426594	total: 445ms	remaining: 267ms
625:	learn: 0.3426265	total: 445ms	remaining: 266ms
626:	learn: 0.3425825	total: 446ms	remaining: 265ms
627:	learn: 0.3425174	total: 447ms	remaining: 264ms
628:	learn: 0.3424254	total: 447ms	remaining: 264ms
629:	learn: 0.3423728	total: 448ms	remaining: 263ms
630:	learn: 0.3423416	total: 448ms	remaining: 262ms
631:	learn: 0.3422881	total: 449ms	remaining: 261ms
632:	learn: 0.3422673	total: 449ms	remaining: 261ms
633:	learn: 0.3421481	total: 450ms	remaining: 260ms
634:	learn: 0.3421113	total: 451ms	remaining: 259ms
635:	learn: 0.3420720	total: 451ms	remaining: 258ms
636:	learn: 0.3419351	total: 452ms	remaining:

,Model,Accuracy,Bal Acc.,Time
1,XGBoost,84.357542,82.951252,0.083429
2,CatBoost,84.357542,82.140975,0.886077
3,AdaBoost,83.798883,82.226614,0.100221
4,Skl GBM,83.798883,81.686430,0.092867
5,Skl HistGBM,83.798883,81.686430,0.096947
6,LightGBM,83.798883,82.226614,0.113231
7,Random Forest,81.564246,80.948617,0.179536
8,Extra Trees,80.446927,78.689065,0.141306
9,Decision Tree,79.888268,78.234519,0.020522


In [523]:
best_model = tree_classifiers["XGBoost"]

# Fit best model with all data
best_model.fit(x,y)
test_pred = best_model.predict(df_test)
df_test = pd.read_csv(DATA_PATH + "test.csv", index_col="PassengerId")
sub = pd.DataFrame(test_pred, index=df_test.index, columns=["Survived"])
sub.head()
sub.to_csv("sub.csv")

/home/umut/.virtualenvs/my-project-env/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[16:02:45] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [525]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

100%|████████████████████████████████████████| 2.77k/2.77k [00:05<00:00, 532B/s]
Successfully submitted to Titanic - Machine Learning from Disaster